In [1]:
import sys
sys.path.insert(0,'/Users/jarad/Fake Folder/Python Libraries')

from jb_libraries import * 
%matplotlib inline

from fab_data import *


== fab_data start ==

match

48 date lines where some date field is null even though the WO is marked as "completed"
WO date nulls even though WO is marked as "completed":

created
0
scheduled
0
active
40
completed
9

Fix these...

wo qty matches
process qty matches
spa qty matches


match
5724 unique WO(s)

match
5339 completed WO(s)

match
141 open WO(s)

match
244 pending WO(s)

your df is called "fab_data_main"


== fab_data end ==



* [Curve Fitting in Python](https://stackoverflow.com/questions/19165259/python-numpy-scipy-curve-fitting)
* [Inputs and Production Functions Chapter 6](https://www.fep.up.pt/docentes/moutinho/micro/Besanko-ch06.pdf)
* [Comparative Statistics](https://en.wikipedia.org/wiki/Comparative_statics)
* [Deriving functions](https://homepage.univie.ac.at/matan.tsur/courses/lecture7.pdf)

In [2]:
pd.read_sql(
'''
SELECT
sku,
sku_id,
sku_name,
sku_price
FROM skus
WHERE sku_name LIKE '%autobagger%' OR sku_name LIKE '%auto bagger%'
ORDER BY sku_price DESC
''', db)

,sku,sku_id,sku_name,sku_price
0,C10815-001,10815,SX Pro Line (1153-02) Auto Bagger,22507.00000
1,C12899-001,12899,Pro-Line SX Sealer Maintenance Kit w/ Tools (for SX 1153 Autobagger),299.95000
2,C12898-001,12898,Pro-Line Seal Wire Support Kit w/ Tools (for SX 1153 Autobagger),129.95000
3,C12684-001,12684,Torsion Spring for Autobagger,68.95000
4,C12900-001,12900,"Stripper Plate Spring Kit, Pro-Line SX (for SX 1153 Autobagger)",27.95000
5,C11075-001,11075,Black thermal Transfer Ribbon for Autobagger,24.67000
6,C12399-001,12399,Teflon tape strip for Autobagger,3.50000
7,C11622-001,11622,"pink anti stat bags 4""x5""x3mil +1"" header w/ sombrero hang hole for SX Pro Line Auto Bagger with...",0.04972
8,C10928-001,10928,"ESD Bag Shield Metal In 5"" x 2-1/2"" for SX Pro Line Auto Bagger (1250 Bags per Roll)",0.03200
9,C10926-001,10926,"pink anti-stat bags 3"" x 4"" x 3mil +1'' header w/sombrero hang hole for SX Pro Line Auto Bagger...",0.02100


### Get data

In [3]:
ab_main = fab_data_main[(fab_data_main['process group'] == 'prep') & (fab_data_main['wo outsourced'] == 'no')].copy()

ab_main.dropna(subset = ['part id'], inplace = True)
ab_main['spa time elapsed'] = ab_main['spa time end'] - ab_main['spa time start']

s1 = ab_main['spa time elapsed'].dt.days * 3600
s2 = ab_main['spa time elapsed'].dt.seconds
ab_main['spa time elapsed (seconds)'] = s1 + s2

# assume a single prepper makes $40,000/yr = 0.00534 per second
l = 0.00534
ab_main['cost of labor'] = ab_main['spa time elapsed (seconds)'] * l

### Label how many autobaggers we have and when

In [4]:
one = pd.to_datetime('2017-12-15 00:00:00')
two = pd.to_datetime('2018-03-15 00:00:00')

ab_main['autobagger'] = np.nan
for index, row in ab_main.iterrows():
    date = row['spa time start']
    if date < one:
        r = 0
    elif date >= one and date < two:
        r = 1
    else:
        r = 2
    ab_main.loc[index, 'autobagger'] = r

### Q: I would like to look at the cost of the autobagger, specifically how much fab work is required to offset the cost.

In [5]:
revenue_main = pd.read_sql(
'''
SELECT
DATE(o.date_purchased) AS date_purchased,
o.orders_id,
op.part_id,
(op.products_quantity - op.products_quantity_free) * op.products_price AS revenue
FROM orders_products op
JOIN orders o ON op.orders_id = o.orders_id
AND DATE(o.date_purchased) >= ' '''+ str(one.date()) +''' '
AND o.orders_status NOT IN (8,9,10,11,12,13,14,15)
''', db)

col_fix(revenue_main)

In [6]:
df1 = revenue_main.groupby('date purchased')[['revenue']].sum()
df1.columns = ['total revenue']

df2 = revenue_main[revenue_main['part id'].isin(main['part id'].tolist())].groupby('date purchased')[['revenue']].sum()
df2.columns = ['total fab revenue']

df3 = revenue_main[revenue_main['part id'].isin(ab_main['part id'].tolist())].groupby('date purchased')[['revenue']].sum()
df3.columns = ['autobagger revenue']

revenue = df1.join(df2).join(df3)

print('proportions of total revenue')
(revenue.iloc[:,1:].div(revenue.iloc[:,0], axis = 0)).mean()

proportions of total revenue


total fab revenue     0.382345
autobagger revenue    0.338649
dtype: float64

In [7]:
revenue.head()

,total revenue,total fab revenue,autobagger revenue
date purchased,,,
2017-12-15,131624.63,32944.86,29818.34
2017-12-16,49662.75,17357.49,16175.39
2017-12-17,46461.14,13802.35,12474.91
2017-12-18,403737.07,144776.40,131617.13
2017-12-19,165748.30,43176.85,41074.63


In [8]:
cost = 22500
r1 = revenue[(revenue.index >= one.date()) & (revenue.index < two.date())]['autobagger revenue'].sum()
r2 = revenue[(revenue.index >= two.date())]['autobagger revenue'].sum()

print('cost of two autobaggers: ${:,.0f}'.format(cost * 2))
print('revenue during first autobagger: ${:,.0f}\nrevenue during second: ${:,.0f}\ncombined revenue: ${:,.0f}'.format(r1,r2,r1+r2))
prof = (r1 + r2) * 0.65
print('profit is ${:,.0f}'.format(prof))
roi = prof/(cost * 2)
print('ROI is {:,.0f}%'.format(roi * 100))

cost of two autobaggers: $45,000
revenue during first autobagger: $3,576,154
revenue during second: $10,110,833
combined revenue: $13,686,987
profit is $8,896,541
ROI is 19,770%


In [9]:
d1 = revenue.index.min()
d2 = revenue.index.max()

print('from {} to {}\nwhich is about {} days'.format(d1,d2,(d2 - d1).days))
per_day = revenue['autobagger revenue'].mean()
print('avg profit per day: ${:,.0f}'.format(per_day * 0.65))
days = np.ceil(cost/(per_day * 0.65))
print('it would take fab about {:,.0f} days to make up the cost of another autobagger'.format(days))

from 2017-12-15 to 2018-11-20
which is about 340 days
avg profit per day: $26,090
it would take fab about 1 days to make up the cost of another autobagger


### Q: ...but for example one of the the autobaggers we have is down this week and I'm wondering like having down time on the autobagger (in this case one week) we have lost X% of potential revenue- and if like having had a backup autobagger which means we would have had two autobaggers running this whole time it would have been worth the cost of having a backup autobagger.

In [10]:
df = ab_main.groupby('autobagger').agg({'spa qty out':'sum',
                                       'cost of labor':'sum',
                                       'part id':'nunique',
                                       'spa time elapsed (seconds)':'sum',
                                       'admin':'nunique'}).rename(columns = {'part id':'unique PN count'})

df['cost per qty'] = df['cost of labor']/df['spa qty out']
df['cost per admin'] = df['cost of labor']/df['admin']

fmt = ['n0','m0',0,'n0','0','m2','m0']

summary = df.copy()
summary.format_(fmt)

,Spa Qty Out,Cost Of Labor,Unique Pn Count,Spa Time Elapsed (Seconds),Admin,Cost Per Qty,Cost Per Admin
autobagger,,,,,,,
0.0,"2,418,246","$302,768",363,"56,698,142",58,$0.13,"$5,220"
1.0,"247,694","$25,034",247,"4,687,971",18,$0.10,"$1,391"
2.0,"605,923","$47,331",321,"8,863,506",22,$0.08,"$2,151"


In [11]:
ab_main['spa date start'] = [pd.to_datetime(x.date()) for x in ab_main['spa time start']]

print('spa qty in per day')
s = '2018-07-01'
print('from {} to {}'.format(s, ab_main['spa date start'].max().date()))
des = ab_main[ab_main['spa date start'] >= pd.to_datetime(s)].groupby('spa date start').agg({'spa qty in':'sum'}).describe()
des.format_(['n0'])

spa qty in per day
from 2018-07-01 to 2018-11-20


,Spa Qty In
count,102
mean,"3,114"
std,"1,150"
min,430
25%,"2,419"
50%,"3,159"
75%,"3,766"
max,"6,358"


In [12]:
days_out = 10
mean = des.loc['mean'].values[0]
increase = np.round(summary.loc[1,'cost per qty'] - summary.loc[2,'cost per qty'],2)

res = (days_out * mean * increase)
print('days out of commission: {} days\nmean prep qty per day: {:,.0f} units\nincrease per unit: ${:,.2f}\ntotal increase in labor-cost: ${:,.0f}'.format(days_out, mean, increase, res))

days out of commission: 10 days
mean prep qty per day: 3,114 units
increase per unit: $0.02
total increase in labor-cost: $623


### Q: I'm mostly curious about our stats the last two weeks (the weeks of Sept 24th and Oct 1st) compared to any two weeks where both autobaggers are up and running, let's say any two weeks between April 2nd and Oct 21st

In [13]:
down = '2018-09-24'
for_t = ab_main.copy()
for_t['spa time elapsed (hours)'] = for_t['spa time elapsed (seconds)']/3600

# get data for after the 2nd autobagger went down
after = for_t[for_t['spa time start'] >= down].groupby('spa date start')[['spa time elapsed (hours)','spa qty in']].sum()
after = after.iloc[0:-1] # exclude the very last day, which is today
n = len(after)

# get data for before, using the same number of days
df = for_t[for_t['spa date start'] < down].copy()
# sort in ascending order, take the last n entries
ds = sorted(list(set(df['spa date start'])))[-n:]

before = for_t[for_t['spa date start'].isin(ds)].groupby('spa date start')[['spa time elapsed (hours)','spa qty in']].sum()

In [14]:
# This is a two-sided test for the null hypothesis that 2 independent samples have identical average (expected) values.

for col in after.columns.tolist():
    b = before[col]
    a = after[col]
    
    if np.abs(b.std() - a.std()) < 1:
        equal_var = True
    else:
        equal_var = False
        
    [t_stat, p_value] = stats.ttest_ind(b,a, equal_var = equal_var)
    p_value = p_value/2
    
    print(col + '\np-value is {:,.2f}'.format(p_value))
    
    if p_value >= 0.05: # reject the null; averages are not the same
        print('the means are statistically different\n')
    else:
        print('the means are not statistically different\n')
        
df1 = pd.DataFrame(columns = ['before'], data = before.mean())
df2 = pd.DataFrame(columns = ['after'], data = after.mean())

res2 = df1.join(df2)
res2['% change'] = res2['after']/res2['before'] - 1
res2.index.name = 'averages'
print('means before and after')
res2.format_(['n2','n2','p2'])

spa time elapsed (hours)
p-value is 0.12
the means are statistically different

spa qty in
p-value is 0.02
the means are not statistically different

means before and after


,Before,After,% Change
averages,,,
spa time elapsed (hours),11.06,12.36,11.68%
spa qty in,"2,737.98","3,208.90",17.20%


In [15]:
print('done')

done
